In [1]:
pip install ipython

Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline

In [3]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import numpy as np
import csv
import os
import tarfile
from urllib.parse import urlparse
from urllib.request import urlretrieve
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
cudnn.benchmark = True
plt.ion()   # interactive mode

In [4]:
import os

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
!pip install -q livelossplot==0.3.3
!pip install -q ipython-autotime
%load_ext autotime

time: 486 µs (started: 2022-07-29 11:16:45 +00:00)


In [7]:
labels = pd.read_csv('../input/ddids/file1.csv')
labels['DDI_file'] = '../input/ddids/ddidiversedermatologyimages/'+labels['DDI_file']
labels.to_csv('file_pp.csv')

time: 28.7 ms (started: 2022-07-29 11:16:45 +00:00)


In [8]:
evalpath = './file_pp.csv'

time: 374 µs (started: 2022-07-29 11:16:45 +00:00)


In [9]:
class_names = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']

def read_object_labels_csv(file,phase, header=True):
    img_t=[]
    dir = './file_pp.csv'
    with open(dir, "r") as f:
        csvReader = csv.reader(f)
        next(csvReader, None)
        k=0
        for line in csvReader:
            k+=1
            image_name= line[2]
            label = line[3:11]
            label = torch.from_numpy(np.array(label, dtype=np.float64))
            item = (image_name,label)
            img_t.append(item)
    return img_t


class DDI(Dataset):
    def __init__(self,phase, csv_path,transform=None):
        self.phase = phase
        self.transform = transform
        file_csv = pd.read_csv(csv_path)
        self.classes = class_names
        self.images = read_object_labels_csv(file_csv,phase)
        print('DDI classification phase={} number of classes={}  number of images={}'.format(phase, len(self.classes), len(self.images)))

    def __getitem__(self, index):
        filename, target = self.images[index]
        filename = filename
        img = Image.open(filename).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        data = {'image':img, 'target': target}
        return data

    def __len__(self):
        return len(self.images)

    def get_number_classes(self):
        return len(self.classes)

time: 2.04 ms (started: 2022-07-29 11:16:45 +00:00)


In [10]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

val_dataset = DDI(transform=data_transforms['val'],phase='Valid',csv_path=evalpath)
image_datasets ={'val': val_dataset}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,shuffle=False, num_workers=2)for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}

DDI classification phase=Valid number of classes=9  number of images=656
time: 12.9 ms (started: 2022-07-29 11:16:45 +00:00)


time: 6.11 s (started: 2022-07-29 11:16:45 +00:00)


In [11]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.6678, 0.5298, 0.5245])
    std = np.array([0.1333, 0.1476, 0.1590])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
data = next(iter(dataloaders['val']))
ps = data['target'].numpy()
l = np.where(ps==1.0)
l = l[1].astype(int)
# Make a grid from batch
out = torchvision.utils.make_grid(data['image'])
imshow(out, title=[class_names[x] for x in l])

In [12]:
model1=models.resnet18(pretrained=False)
model1.fc=nn.Sequential(nn.Linear(512, 8), nn.Sigmoid())
model1.to(device)
checkpoint = torch.load('../input/resnet18isic/RESNET18_wt_epoch_11_batch_312_auc_0.9114.pt')
model1.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint

time: 4.41 s (started: 2022-07-29 11:16:52 +00:00)


In [13]:
model2=models.resnet18(pretrained=False)
model2.fc=nn.Sequential(nn.Linear(512, 8), nn.Sigmoid())
model2.to(device)
checkpoint = torch.load('../input/resnet18isic/RESNET18_wt_epoch_13_batch_312_auc_0.9108.pt')
model2.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint

time: 1.15 s (started: 2022-07-29 11:16:56 +00:00)


In [14]:
model3=models.resnet18(pretrained=False)
model3.fc=nn.Sequential(nn.Linear(512, 8), nn.Sigmoid())
model3.to(device)
checkpoint = torch.load('../input/resnet18isic/RESNET18_wt_epoch_15_batch_312_auc_0.9112.pt')
model3.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint

time: 1.06 s (started: 2022-07-29 11:16:57 +00:00)


In [15]:
model4=models.resnet18(pretrained=False)
model4.fc=nn.Sequential(nn.Linear(512, 8), nn.Sigmoid())
model4.to(device)
checkpoint = torch.load('../input/resnet18isic/RESNET18_wt_epoch_17_batch_312_auc_0.9055.pt')
model4.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint

time: 1.23 s (started: 2022-07-29 11:16:58 +00:00)


In [16]:
model5=models.resnet18(pretrained=False)
model5.fc=nn.Sequential(nn.Linear(512, 8), nn.Sigmoid())
model5.to(device)
checkpoint = torch.load('../input/resnet18isic/RESNET18_wt_epoch_9_batch_312_auc_0.9091.pt')
model5.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint

time: 1.1 s (started: 2022-07-29 11:17:00 +00:00)


In [17]:
from tqdm._tqdm_notebook import tqdm_notebook
def compute_predictions(model):
    model.eval()
    pred_lst=[]
    gt_lst=[]
    for i, sample in enumerate(dataloaders['val']):    
        images=sample['image']
        labels=sample['target']
        del sample
        images = images.to(device)
        labels=labels.numpy()
        outputs=model(images)
        outputs=outputs.detach()
        outputs=outputs.cpu()
        outputs=outputs.numpy()
        pred_lst.append(outputs)
        gt_lst.append(labels)
        del labels, outputs, images

    pred_lst=np.concatenate(pred_lst, axis=0)
    gt_lst=np.concatenate(gt_lst, axis=0)
    auc_lst=[]
    acc_lst=[]
    for k in range(0,8):
        tmp_gt=gt_lst[:, k]
        tmp_pred=pred_lst[:,k]
        tmp_acc=accuracy_score(tmp_gt, tmp_pred.round())
        acc_lst.append(tmp_acc)
        tmp_auc=roc_auc_score(tmp_gt, tmp_pred)
        auc_lst.append(tmp_auc)

    auc_lst=np.array(auc_lst)
    acc_lst=np.array(acc_lst)
    print('\n')
    print(auc_lst)
    print('\n')
    print(acc_lst)
    auc=np.mean(auc_lst)
    acc = np.mean(acc_lst)
    print(auc)
    print(acc)
    f1s=0.0
    return auc,acc,pred_lst,gt_lst

time: 3.13 ms (started: 2022-07-29 11:17:01 +00:00)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.


In [18]:
auc1,acc1,pred_lst1,gt_lst1=compute_predictions(model1)



[0.44506937 0.57978499 0.58138054 0.62193252 0.50165608 0.47132205
 0.51086843 0.59917394]


[0.76829268 0.5304878  0.91615854 0.99237805 0.875      0.96646341
 0.97560976 0.91920732]
0.5388984897278326
0.8679496951219512
time: 16.6 s (started: 2022-07-29 11:17:01 +00:00)


In [19]:
auc1

0.5388984897278326

time: 3.37 ms (started: 2022-07-29 11:17:17 +00:00)


In [20]:
acc1

0.8679496951219512

time: 2.17 ms (started: 2022-07-29 11:17:17 +00:00)


In [21]:
pred_lst1.shape

(656, 8)

time: 2.67 ms (started: 2022-07-29 11:17:17 +00:00)


In [22]:
gt_lst1.shape

(656, 8)

time: 2.85 ms (started: 2022-07-29 11:17:17 +00:00)


In [23]:

print('Computing Test Predictions')
auc1,acc1,pred_lst1,gt_lst1 = compute_predictions(model1)
auc2,acc2,pred_lst2,gt_lst2 = compute_predictions(model2)
auc3,acc3,pred_lst3,gt_lst3 = compute_predictions(model3)
auc4,acc4,pred_lst4,gt_lst4 = compute_predictions(model4)
auc5,acc5,pred_lst5,gt_lst5 = compute_predictions(model5)



Computing Test Predictions


[0.44506937 0.57978499 0.58138054 0.62193252 0.50165608 0.47132205
 0.51086843 0.59917394]


[0.76829268 0.5304878  0.91615854 0.99237805 0.875      0.96646341
 0.97560976 0.91920732]
0.5388984897278326
0.8679496951219512


[0.45099363 0.58868911 0.57357998 0.62039877 0.48563975 0.46400918
 0.55943838 0.59448043]


[0.73932927 0.54573171 0.91006098 0.99237805 0.86890244 0.96646341
 0.97560976 0.91768293]
0.5421536517245761
0.8645198170731707


[0.48248969 0.588783   0.58176261 0.57592025 0.47849365 0.51182965
 0.58647946 0.58108827]


[0.76829268 0.54115854 0.91006098 0.99085366 0.86280488 0.96646341
 0.97713415 0.91463415]
0.5483558213883448
0.8664253048780488


[0.5072366  0.58194451 0.57803744 0.63458589 0.46592559 0.46451104
 0.57379095 0.55245784]


[0.76371951 0.5152439  0.91006098 0.99085366 0.86128049 0.96646341
 0.97713415 0.91463415]
0.544811232052582
0.8624237804878049


[0.45691789 0.57541899 0.56514264 0.53757669 0.51583485 0.49197017
 0.472490

In [24]:
pred_lst = (pred_lst1+pred_lst2+pred_lst3+pred_lst4+pred_lst5)/5
gt_lst = gt_lst1
auc_lst=[]
acc_lst=[]
for k in range(0,8):
    tmp_gt=gt_lst[:, k]
    tmp_pred=pred_lst[:,k]
    tmp_acc=accuracy_score(tmp_gt, tmp_pred.round())
    acc_lst.append(tmp_acc)
    tmp_auc=roc_auc_score(tmp_gt, tmp_pred)
    auc_lst.append(tmp_auc)

auc_lst=np.array(auc_lst)
acc_lst=np.array(acc_lst)
print('Printing AUC LIST \n')
print(auc_lst)
print('Printing ACC LIST\n')
print(acc_lst)
auc=np.mean(auc_lst)
acc = np.mean(acc_lst)
print(auc)
print(acc)

Printing AUC LIST 

[0.46194226 0.58779713 0.57300688 0.59202454 0.48861162 0.47132205
 0.53905356 0.58102569]
Printing ACC LIST

[0.75914634 0.53506098 0.91463415 0.99237805 0.87652439 0.96646341
 0.97713415 0.91920732]
0.536847965530224
0.8675685975609756
time: 16.7 ms (started: 2022-07-29 11:18:14 +00:00)


In [25]:
acc

0.8675685975609756

time: 2.31 ms (started: 2022-07-29 11:18:14 +00:00)


In [26]:
auc

0.536847965530224

time: 2.08 ms (started: 2022-07-29 11:18:14 +00:00)
